# Local Learning for benchmark models
This includes: 
- **Dense**: 3 layers, with 16 units and relu activation function
- **Bidirectional LSTM**: 2 bidirectional layers, with 20 LSTM cells and tanh 

In [7]:
#Imports
import pandas as pd
import os
import tensorflow as tf

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

In [ ]:
#Get data 
cwd = os.path.normpath(os.path.dirname(os.getcwd()))
df = pd.read_csv(cwd+'/data/2feature_engineering_data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

num_user = 3
df_array = []
for idx in range(num_user):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


# Dense Model

In [ ]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [5]:
#Dense 1 -------------------------------------------------------------

#Dense Hyperparameter
dense_architecture = "L3_U16"
dense_layers = 3
dense_units = 16
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        dense_model = m1.build_dense_model(X_train[f'user{idx+1}'], horizon, num_layers=dense_layers, units=dense_units, batch_size=batch_size)
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

    dense_model.save(cwd + f"/models/Local_learning/Dense/{dense_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  


for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

User:  1
INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User1\assets


Saved Soft_Dense_MoE
User:  3
INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User2\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User2\assets


Saved Soft_Dense_MoE
User:  4
INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning/models/Local_learning/Dense/L3_U16/User3\assets


Saved Soft_Dense_MoE
User:  5


KeyboardInterrupt: 

In [ ]:
dense_results.to_csv(f'evaluations/Local_learning/Dense_benchmark.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,L3_U16,9.349461,0.377300,0.034101,0.001172,0.184646,0.003174,7.684803e+04,76368.551047,0.128049,0.003624
1,L3_U16,6.901573,0.381778,0.017999,0.000795,0.134140,0.002948,1.563642e+05,19373.125474,0.078911,0.003110
2,L3_U16,9.861912,0.370500,0.021385,0.000986,0.146210,0.003351,5.275009e+04,27010.966959,0.083458,0.003016
3,L3_U16,12.407765,0.375891,0.022548,0.000398,0.150155,0.001323,1.323894e+05,10105.458214,0.065013,0.000794
4,L3_U16,8.969604,0.378942,0.014485,0.000220,0.120351,0.000911,1.006256e+05,8686.180798,0.077778,0.003253
...,...,...,...,...,...,...,...,...,...,...,...
85,L5_U32,10.056733,0.486687,0.026088,0.001580,0.161467,0.004935,1.361541e+05,56963.783291,0.106361,0.006344
86,L5_U32,10.854706,0.491758,0.019055,0.000521,0.138033,0.001895,1.674367e+05,30052.191224,0.094163,0.000811
87,L5_U32,7.987520,0.505900,0.011377,0.000051,0.106665,0.000239,5.761627e+04,2624.610136,0.045686,0.000787
88,L5_U32,9.432113,0.494297,0.017981,0.000842,0.134068,0.003128,5.235218e+05,109637.100059,0.096859,0.005100


# Model Bidirectional LSTM benchmark

In [ ]:
#LSTM Model 1 ------------------------------------------------------------

#Dense Hyperparameter
lstm_architecture = "L1_U8"
lstm_layers = 1
lstm_units = 8
lstm_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        lstm_model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
        lstm_histroy, lstm_user_results = mh.compile_fit_evaluate_model(
            model=lstm_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=lstm_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        lstm_all_results = pd.merge(lstm_all_results, lstm_user_results, how='outer')   

    lstm_model.save(cwd + f"/models/Local_learning/LSTM/{lstm_architecture}/User{idx}")
    print("Saved Soft_Dense_MoE")  


for idx in range(len(df_array)):
    new_row = {
        'architecture': lstm_architecture,
        'train_time': lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : lstm_all_results[lstm_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row


User:  1
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U8/User0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U8/User0\assets


Saved Soft_Dense_MoE
User:  2
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U8/User1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U8/User1\assets


Saved Soft_Dense_MoE
User:  3
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U8/User2\assets


...

Saved Soft_Dense_MoE
User:  29
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U20/User28\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U20/User28\assets


Saved Soft_Dense_MoE
User:  30
INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U20/User29\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/Local_learning/LSTM/L2_U20/User29\assets


Saved Soft_Dense_MoE


In [ ]:
dense_results.to_csv(f'evaluations/Local_learning/LSTM_benchmark.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,L3_U16,9.349461,0.377300,0.034101,0.001172,0.184646,0.003174,7.684803e+04,76368.551047,0.128049,0.003624
1,L3_U16,6.901573,0.381778,0.017999,0.000795,0.134140,0.002948,1.563642e+05,19373.125474,0.078911,0.003110
2,L3_U16,9.861912,0.370500,0.021385,0.000986,0.146210,0.003351,5.275009e+04,27010.966959,0.083458,0.003016
3,L3_U16,12.407765,0.375891,0.022548,0.000398,0.150155,0.001323,1.323894e+05,10105.458214,0.065013,0.000794
4,L3_U16,8.969604,0.378942,0.014485,0.000220,0.120351,0.000911,1.006256e+05,8686.180798,0.077778,0.003253
...,...,...,...,...,...,...,...,...,...,...,...
175,L2_U20,53.499830,2.639695,0.030621,0.003712,0.174779,0.010494,1.177978e+05,38501.550518,0.110774,0.009411
176,L2_U20,109.373161,2.498577,0.020880,0.000204,0.144498,0.000706,1.224330e+05,5239.017740,0.103547,0.000877
177,L2_U20,68.815570,2.540393,0.010702,0.000358,0.103442,0.001724,5.429569e+04,13804.586163,0.046831,0.004418
178,L2_U20,56.531725,2.599237,0.020039,0.002462,0.141384,0.008588,5.313414e+05,78721.016695,0.100944,0.003674


In [ ]:
dense_results[-2:]

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
178,L2_U20,56.531725,2.599237,0.020039,0.002462,0.141384,0.008588,531341.375,78721.016695,0.100944,0.003674
179,L2_U20,61.471449,2.553378,0.019009,0.000366,0.137867,0.001324,1025715.125,119481.185859,0.087026,0.004663


In [ ]:
print("L3_U16 Dense: ", dense_results[0:29]["mse"].mean())
print("L1_U4 Dense: ", dense_results[0:29]["mse"].mean())
print("L5_U32 Dense: ", dense_results[0:29]["mse"].mean())
print("L2_U8 BiLSTM: ", dense_results[30:59]["mse"].mean())
print("L1_U8 BiLSTM: ", dense_results[60:89]["mse"].mean())
print("L2_U20 BiLSTM: ", dense_results[90:119]["mse"].mean())

L3_U16 Dense:  0.018908821784987533
L1_U4 Dense:  0.018908821784987533
L5_U32 Dense:  0.018908821784987533
L2_U8 BiLSTM:  0.019713866615389614
L1_U8 BiLSTM:  0.019076576306559574
L2_U20 BiLSTM:  0.020098986157655034
